Creating an etl Job

In [ ]:
from spark_session_factory import create_spark_session
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import DecimalType



In [ ]:
def run_etl(spark: SparkSession, input_path_topic1: str, input_path_topic2: str, output_path: str):
    """
    Main ETL pipeline: read -> transform -> write.
    
    Args:
        spark: Active SparkSession
        input_path: Landing zone path (e.g., '/opt/spark-data/landing/*.json')
        output_path: Gold zone path (e.g., '/opt/spark-data/gold')
    """
    # TODO: Implement
    # Accessing Spark Session
    spark = spark
    
    # Reading from the two json files and cache them for more use
    df_topic_transaction = spark.read.json(input_path_topic1).cache()
    df_topic_user = spark.read.json(input_path_topic2).withColumnRenamed("timestamp","u_timestamp").cache()
    
    # df_topic_transaction.show(truncate=False)
    # df_topic_user.show(truncate=False)
    
    # print(df_topic_transaction.printSchema())
    # print(df_topic_user.printSchema())
    
    # Each product gets its own row called product and renaming timestamp
    df_exploded_transaction = df_topic_transaction.withColumn("product", F.explode("products")) \
        .withColumnRenamed("timestamp", "t_timestamp")
        
    # df_exploded_transaction.select("product.product_id").show(truncate= False)
    
    # print(df_exploded_transaction.printSchema())
    # print(df_topic1.select("*").filter(F.col("user_id") == '10f1bc81').count())
    # print(df_topic2.select('*').filter(F.col("user_id") == '10f1bc81').count())
    
    
    # Join on user_id and the product_id being the same
    # Might lead to some data explosion but best bet to limit that
    df_topics_combined = df_exploded_transaction.alias("transaction") \
        .join(df_topic_user.alias("user"), 
              ((F.col("user.user_id") == F.col("transaction.user_id")) & (F.col("user.product_id") == F.col("transaction.product.product_id"))), 
              "inner")
    
    # df_topics_combined.show(truncate=False)
    
    # Split up the columns with more items in it like address, or the products itself
    df_advertising_dept = df_topics_combined \
        .select("products","browser",
            "transaction.user_id","t_timestamp", "device", 
            "shipping_address.state", "shipping_address.city", "shipping_address.country") \
                .withColumn("product", F.explode("products")).drop("products")
    
    # Collecting each name of the product row
    prod_column_names = df_advertising_dept.select("product").schema["product"].dataType.__dict__["names"]
    
    # df_advertising_dept.printSchema()
    
    # Create a new column for each product_id,Name,Quantity, and unit_price. Also solves for the line revenue
    df_advertising_dept = df_advertising_dept.select("*",*[F.col(f"product.{x}").alias(x) for x in prod_column_names]).drop("product") \
        .withColumn("line_revenue", F.round(F.col("quantity")*F.col("unit_price"),2).cast(DecimalType(10,2))) \

        
    

    # print(df_topic_user.count())
    # print(df_topic_transaction.count())
    # print(df_topics_combined.show(truncate=False))
    
    
    # print(df_advertising_dept.printSchema())
    # print(df_advertising_dept.show(truncate=False))
    

    # The tables we need to write to the gold layer
    df_advertising_dept
    
    
    spark.stop() 
    pass


Questions we want to answer!

We can join on by users,
We can rank devices most likely to spend the most

In [186]:
run_etl(create_spark_session("ETL_job"),'../data/landing/transaction_events_1767986916.1293132.json', '../data/landing/user_events_1767986960.934074.json', "")

+----------+
|product_id|
+----------+
|PROD_1078 |
|PROD_1069 |
|PROD_1012 |
|PROD_1140 |
|PROD_1082 |
|PROD_1060 |
|PROD_1024 |
|PROD_1181 |
|PROD_1105 |
|PROD_1082 |
|PROD_1045 |
|PROD_1157 |
|PROD_1138 |
|PROD_1140 |
|PROD_1089 |
|PROD_1161 |
|PROD_1082 |
|PROD_1130 |
|PROD_1041 |
|PROD_1180 |
+----------+
only showing top 20 rows

root
 |-- browser: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- t_timestamp: string (nullable = true)
 |-- device: string (nullable = true)
 |-- state: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- category: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- quantity: long (nullable = true)
 |-- unit_price: double (nullable = true)
 |-- line_revenue: decimal(10,2) (nullable = true)

None
+-------+--------+---------------------------+-------+-----+--------------+-------+-----------+----------+-----------------